In [ ]:
import os
import sys
import importlib

# Thêm đường dẫn cho các module
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'import'))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'module'))

# Import các module
import import_default
import import_database  
import import_other
import clean_data
import candle_chart

importlib.reload(import_default)
importlib.reload(import_database)
importlib.reload(import_other)
importlib.reload(clean_data)
importlib.reload(candle_chart)

from import_default import *
from import_database import *
from import_other import *
from clean_data import *
from candle_chart import *

#### Lựa chọn cổ phiếu phân tích

In [2]:
main_stock = 'HPG'

In [ ]:
market_period_dict = {
    "period_1": {
        "name": "Sóng hồi phục từ đáy COVID-19",
        "start_date": "2020-04-01",
        "end_date": "2020-07-29",
        "trend": "uptrend",
        "phase": 1
    },
    "period_2": {
        "name": "Sóng tăng vượt đỉnh tiền COVID-19",
        "start_date": "2020-07-29",
        "end_date": "2021-01-29",
        "trend": "uptrend",
        "phase": 2
    },
    "period_3": {
        "name": "Sóng tăng thứ hai sau COVID-19",
        "start_date": "2021-01-29",
        "end_date": "2021-07-20",
        "trend": "uptrend",
        "phase": 3
    },
    "period_4": {
        "name": "Sóng tăng lên đỉnh lịch sử 1500 điểm",
        "start_date": "2021-07-20",
        "end_date": "2022-04-04",
        "trend": "uptrend",
        "phase": 4
    },
    "period_5": {
        "name": "Nhịp điều chỉnh từ đỉnh 1500 điểm",
        "start_date": "2022-04-04",
        "end_date": "2022-07-11",
        "trend": "downtrend",
        "phase": 1
    },
    "period_6": {
        "name": "Sóng hồi trong xu hướng giảm 2022",
        "start_date": "2022-07-11",
        "end_date": "2022-09-05",
        "trend": "downtrend",
        "phase": 2
    },
    "period_7": {
        "name": "Nhịp giảm tạo đáy xu hướng 2022",
        "start_date": "2022-09-05",
        "end_date": "2022-11-16",
        "trend": "downtrend",
        "phase": 3
    },
    "period_8": {
        "name": "Sóng tăng mạnh từ đáy 873 điểm",
        "start_date": "2022-11-16",
        "end_date": "2022-12-06",
        "trend": "uptrend",
        "phase": 1
    },
    "period_9": {
        "name": "Giai đoạn tích lũy sau phục hồi",
        "start_date": "2022-12-06",
        "end_date": "2023-04-26",
        "trend": "sideways",
        "phase": 1
    },
    "period_10": {
        "name": "Sóng tăng chính năm 2023",
        "start_date": "2023-04-26",
        "end_date": "2023-09-12",
        "trend": "uptrend",
        "phase": 1
    },
    "period_11": {
        "name": "Nhịp điều chỉnh mạnh cuối năm 2023",
        "start_date": "2023-09-12",
        "end_date": "2023-11-01",
        "trend": "downtrend",
        "phase": 1
    },
    "period_12": {
        "name": "Sóng tăng cuối 2023 - đầu 2024",
        "start_date": "2023-11-01",
        "end_date": "2024-03-08",
        "trend": "uptrend",
        "phase": 1
    },
    "period_13": {
        "name": "Giai đoạn Sideway kéo dài (2024)",
        "start_date": "2023-11-01",
        "end_date": "2025-01-13",
        "trend": "sideways",
        "phase": 1
    },
    "period_14": {
        "name": "Sóng tăng đầu 2025 (trước thông tin thuế)",
        "start_date": "2025-01-13",
        "end_date": "2025-03-25",
        "trend": "uptrend",
        "phase": 1
    },
    "period_15": {
        "name": "Giảm mạnh do thông tin thuế",
        "start_date": "2025-03-25",
        "end_date": "2025-04-09",
        "trend": "downtrend",
        "phase": 1
    },
    "period_16": {
        "name": "Sóng phục hồi sau thông tin thuế",
        "start_date": "2025-04-09",
        "end_date": "2025-06-10",
        "trend": "uptrend",
        "phase": 1
    },
    "period_17": {
        "name": "Sóng tăng vượt đỉnh",
        "start_date": "2025-06-10",
        "end_date": "2025-07-23",
        "trend": "uptrend",
        "phase": 2
    }
}


#### Đọc dữ liệu từ Mongo

In [3]:
full_stock_classification_df = get_mongo_collection(ref_db, 'full_stock_classification')
current_quarter_classification_df = get_mongo_collection(ref_db, 'current_quarter_classification')

#### Đọc và làm sạch dữ liệu từ FiinProX

In [4]:
raw_df_dict = {}
raw_df_dict['ptcb'] = pd.read_excel('../data/raw/raw_ptcb.xlsx', skiprows=7, skipfooter=11)
raw_df_dict['bctc'] = pd.read_excel('../data/raw/raw_bctc.xlsx', skiprows=7, skipfooter=11)
raw_df_dict['current'] = pd.read_excel('../data/raw/raw_current.xlsx', skiprows=7, skipfooter=11)

for key, df in raw_df_dict.items():
    temp_df = df[df.iloc[:, 0].isin(full_stock_classification_df['ticker'].tolist())].reset_index(drop=True)
    raw_df_dict[key] = temp_df

# Tìm ra danh sách cổ phiếu chung cho cả 3 bảng
common_tickers = set(raw_df_dict['ptcb'].iloc[:, 0])
for key, df in raw_df_dict.items():
    common_tickers = common_tickers.intersection(set(df.iloc[:, 0]))

# Lọc tất cả các DataFrame để chỉ bao gồm các mã cổ phiếu chung
for key, df in raw_df_dict.items():
    raw_df_dict[key] = df[df.iloc[:, 0].isin(common_tickers)].reset_index(drop=True)

In [5]:
clean_df_dict = {}
for key, df in raw_df_dict.items():
    temp_df = transform_to_long_format(df)
    temp_df.to_csv(f'../data/clean/clean_{key}.csv', index=False)
    clean_df_dict[key] = temp_df

#### Phân tích BCTC trong 4 năm gần nhất

In [6]:
fs_dict = get_financial_statements(
    stock='HPG',
    year='2024', 
    quarter='4',
    period_count='16'
)

#### So sánh các chỉ số cơ bản với ngành

#### Phân tích kĩ thuật

In [7]:
projection = {
    "_id": 0,
    "date": 1,
    "ticker": 1,
    "open": 1,
    "high": 1,
    "low": 1,
    "close": 1,
    "pct_change": 1,
    "diff": 1,
    "volume": 1,
    "option": 1,
    "SMA_20": 1,
    "SMA_60": 1,
    "RSI_14": 1,
    "week_open": 1,
    "month_prev_high": 1,
    "month_prev_low": 1,
    "month_open": 1,
    "quarter_prev_high": 1,
    "quarter_prev_low": 1,
    "quarter_open": 1,
    "year_open": 1,
    "MFIBO_0382": 1,
    "MFIBO_0500": 1,
    "MFIBO_0618": 1,
    "QFIBO_0382": 1,
    "QFIBO_0500": 1,
    "QFIBO_0618": 1,
	"YFIBO_0382": 1,
    "YFIBO_0500": 1,
    "YFIBO_0618": 1,
}


today_stock_df = get_mongo_collection(stock_db, "today_stock", find_query={"ticker": main_stock}, projection=projection)
history_stock_df = get_mongo_collection(stock_db, "history_stock", find_query={"ticker": main_stock}, projection=projection)
full_stock_ta_df = pd.concat([today_stock_df, history_stock_df], axis=0, ignore_index=True).iloc[:120]

In [8]:
image_name = f'TA_{main_stock}.png'

line_name_dict = {
    'SMA_20': 'SMA 20',
    'SMA_60': 'SMA 60',
    'week_open': 'WEEK OPEN',
    'month_open': 'MONTH OPEN',
    'quarter_open': 'QUARTER OPEN',
    'year_open': 'YEAR OPEN',
    'month_prev_high': 'LAST MHIGH',
    'month_prev_low': 'LAST MLOW',
    'quarter_prev_high': 'LAST QHIGH',
    'quarter_prev_low': 'LAST QLOW',
    'MFIBO_0382': 'MFIBO 0.382',
    'MFIBO_0500': 'MFIBO 0.500',
    'MFIBO_0618': 'MFIBO 0.618',
    'QFIBO_0382': 'QFIBO 0.382',
    'QFIBO_0500': 'QFIBO 0.500',
    'QFIBO_0618': 'QFIBO 0.618',
	'YFIBO_0382': 'YFIBO 0.382',
    'YFIBO_0500': 'YFIBO 0.500',
    'YFIBO_0618': 'YFIBO 0.618'
}

chart_config = create_chart_config(
	title_font_size=30,
	axis_font_size=24,
	tag_font_size=24,
	price_tag_font_size=24,
    min_spacing_ratio = 0.042,
	margin=dict(l=20, r=220, t=20, b=20, pad=10)
)

max_attempts = 5
for attempt in range(max_attempts):
    try:
        fig, fig_images = create_financial_chart(
            full_stock_ta_df.sort_values('date'), # DataFrame của bạn
            width=1400,
            height=1200,
            line_name_dict=line_name_dict,
            line_columns=list(line_name_dict.keys()),
            chart_config=chart_config,
            symbol_name=main_stock,
            path='../output',
            image_name = image_name
        )
        break
    except Exception as e:
        if attempt == max_attempts - 1:
            raise e
# fig.show()